# Lab 14: Multi-Object Photometry
This lab will demonstrate how to use our photometry tools on an entire image.

In [ ]:
# initial imports
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import scipy.stats
from astropy.stats import sigma_clipped_stats
import astropy.units as u

# change some default plotting parameters
import matplotlib as mpl
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.interpolation'] = 'nearest'
mpl.rcParams['image.cmap'] = 'Greys_r'

# run the %matplotlib magic command to enable inline plotting
# in the current Notebook
%matplotlib inline

## Load our Image

In [ ]:
#Read in our image
hdulist = fits.open('data/aa_aql0007.fits')
hdulist.info()
prihdr = hdulist[0].header
image_data = hdulist[0].data.astype(np.float) *u.adu #Ensure a good data type and units
image_data = image_data[0:600,0:600] #Shrink our image
print(np.median(image_data))

In [ ]:
#Get Useful header info
read_noise = np.float(prihdr['rdnoise']) *u.electron
gain = np.float(prihdr['gain']) * u.electron / u.adu
exptime = np.float(prihdr['exptime']) * u.s
print(read_noise)
print(gain)
print(exptime)

In [ ]:
#Change into electrons
image_data = image_data * gain

In [ ]:
#We also need to know the dark current.
dark = fits.open('data/Dark.fits')
dark_hdr = dark[0].header
dark_data = dark[0].data *u.adu * gain
dark_current = np.median(dark_data/(dark_hdr['exptime']*u.s))
print(dark_current)
dark.close() #Save some memory

## Source Detection
The main issue with doing photometry on an entire image is finding all the sources. There are a number of ways to do this, but we will use a standard routine called DAOFIND. DAOFIND runs on images that have had their background removed. We also need to know the standard FWHM of a star in the image.

## FWHM
We need to know what the full-width at Half Maximimum is for a generic star on our image. To do that we can extract a small image around a bright isolated star and then do some basic fitting on it. Let's pick the star located at (503,440).

In [ ]:
star_data = image_data[420:460,483:523] #Don't forget to reverse x and y
from astropy.visualization import ZScaleInterval
interval = ZScaleInterval()
(imin,imax) = interval.get_limits(star_data)
plt.imshow(star_data, vmin=imin,vmax=imax)
plt.colorbar()

In [ ]:
from photutils import fit_2dgaussian
gfit = fit_2dgaussian(star_data.value) #fit_2dgaussian can't handle quantities
ave_sigma = np.mean([gfit.x_stddev.value,gfit.y_stddev.value])
ave_fwhm = ave_sigma * 2.355
print(ave_fwhm)

## Robust Statistics
We are interested in the mean, median, and standard deviation of the background, but we don't really want the stars to be contributing to this. One way to solve this issue is sigma clipping. In sigma clipping we calculate a median and standard deviation and throw out data points more than some number of sigma away from the median. This process continues for some number of iterations.

In [ ]:
mean, median, std = sigma_clipped_stats(image_data, sigma=3.0, maxiters=5)
print(mean,median,std)

## DAOfind
Let's run DAOFind on the whole image. Note you can change the `5` in the threshold value as appropriate for finding your targets.

In [ ]:
from photutils import DAOStarFinder
daofind = DAOStarFinder(fwhm=ave_fwhm, threshold=5.*std.value) #daofind doesn't like quantities    
sources = daofind(image_data - median)
sources

## Aperture Photometry
Now we just follow our procedure from Lab 13, but use apertures based on the FWHM. I have chosen 2 x FWHM for my star aperture, and 2.5 x FWHM for my sky inner annulus radius. The choice of these numbers will vary depending on your data, and it is always a balance between gettting enough star light, whicl

In [ ]:
from photutils import CircularAperture, CircularAnnulus, aperture_photometry
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=ave_fwhm*2)
bkg_apertures = CircularAnnulus(positions, r_in=ave_fwhm*2.5, r_out=ave_fwhm*2.5 + 5)

In [ ]:
(imin,imax) = interval.get_limits(image_data)
plt.imshow(image_data, vmin=imin,vmax=imax)
plt.colorbar()
apertures.plot(color='blue')
bkg_apertures.plot(color='cyan', hatch='//', alpha=0.8)
#Add some Ids
for i, id in enumerate(sources['id']):
    plt.annotate(id, (sources['xcentroid'][i],sources['ycentroid'][i]),color='gold')
plt.savefig('data/apertures.png',dpi=150) #Save image as a file

In [ ]:
#It turns out that the best measurement of the backgroud for photometry is not the mean, but rather the mode
def getBackground(image_data, apertures):
    bkgs = list()
    for m in apertures.to_mask('center'):
        inc_pix = m.cutout(image_data)[m.cutout(image_data) *m.data >0]        
        mode_bak = scipy.stats.mode(inc_pix)
        bkgs.append(float(mode_bak[0]))
    return np.array(bkgs)*u.electron

In [ ]:
#Get Apeture Sums
phot = aperture_photometry(image_data, apertures)
bphot = aperture_photometry(image_data, bkg_apertures)
bkg_modes = getBackground(image_data,bkg_apertures)    
bkg_sums = bkg_modes * apertures.area
phot['bkg_sum'] = bkg_sums
# subtract the background
flux_bkgsub = phot['aperture_sum'] - bkg_sums
# Any stars that have a negative flux_bkgsub are probably just background, so set it to np.nan
flux_bkgsub[(flux_bkgsub.value < 1)] = np.nan
phot['aperture_sum_bkgsub'] = flux_bkgsub

In [ ]:
#Calculate Error
a_b = 1 + (apertures.area/bkg_apertures.area)
Nvariance = phot['aperture_sum_bkgsub']*u.electron + apertures.area*a_b*(
    bkg_modes*u.electron+exptime*dark_current*u.electron + read_noise**2)
Nsigma = np.sqrt(Nvariance)
phot['aperture_sum_bkgsub_err'] = Nsigma

In [ ]:
zpt = 25.
mag = zpt - 2.5*np.log10(phot['aperture_sum_bkgsub'].value) + 2.5*np.log10(exptime.value)
merr = 1.0857 *phot['aperture_sum_bkgsub_err']/phot['aperture_sum_bkgsub']
phot['Mag'] = mag * u.mag
phot['Mag_err'] = merr * u.mag
phot

## Look at your data
Due to crowding or being near the edge of the image, some stars are probably not useful. Look at your apertures.png file to see which stars are likely not to be good.

![](data/apertures.png)

## Save your data
The last step is to save your data, so that you can use it later. All of the non-image data we work with are astropy tables. You can use the `write()` method to write your data out.

In [ ]:
phot.write('data/phot.csv',overwrite=True)